STS , NLI for competition wiht NIA, KEIRS

#1. NLI 학습
#2. STS 학습
#3. NLI + sts 학습 -> 논문에서 가장 좋은 점수가 나온다고 함

※시간 및 컴퓨터 자원 제한으로 NLI만 학습

In [1]:
#dir setting
import os
os.chdir('/content/drive/MyDrive/Nia_competition')

In [2]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 4.9 MB 19.1 MB/s 
     |████████████████████████████████| 1.3 MB 41.6 MB/s 
     |████████████████████████████████| 163 kB 52.8 MB/s 
     |████████████████████████████████| 6.6 MB 48.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4dc388c5d7802e4d6c9ddf8f33c2146f32d0169fc5f9ff03e95edb63bf9be33a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
#for specific domain
# import packages
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, util, InputExample, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import math
import numpy as np

In [4]:
#kpfbert is built by Korea news data
model_name = "jinmang2/kpfbert"

In [5]:
embedding_model = models.Transformer(model_name,
                                     max_seq_length = 512,
                                     do_lower_case = True)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/454M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at jinmang2/kpfbert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/335 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/276k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
label_dict = {"contradiction": 0, "entailment": 1, "neutral": 2}
train_data = []

In [7]:
#data for training
#inputexampe -> 두 개의 문장 쌍과 라벨을 묶음. -> 모델 학습할 수 있는 형태로 변환
with open('/content/drive/MyDrive/Nia_competition/snli_1.0_train.ko.tsv', "rt", encoding="utf-8") as fIn:
    lines = fIn.readlines()
    for line in lines:
        try:
          s1, s2, label = line.split('\t')
          label = label_dict[label.strip()]
        except ValueError as e : pass
        train_data.append(InputExample(texts=[s1, s2], label=label))

In [8]:
train_dataset = SentencesDataset(train_data, model=model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

In [9]:
dev_data = []
with open('/content/drive/MyDrive/Nia_competition/tune_dev.tsv', 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
        dev_data.append(InputExample(texts= [s1,s2], label=score))

In [10]:
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_data, batch_size=16, name='sts-dev')

In [ ]:
#hypter parmas setting
epochs = 5
train_batch_size = 16
model_save_path = '/content/drive/MyDrive/Nia_competition/training_model_sts-'

from sentence_transformers import losses

loss = losses.SoftmaxLoss(model = model, sentence_embedding_dimension = model.get_sentence_embedding_dimension(), num_labels=len(label_dict))
#linear learning-rate warmup steps
warmup_steps = math.ceil(len(train_dataset) * epochs / train_batch_size * 0.1) #10% of train data for warm-up

#train
model.fit(
    train_objectives = [(train_dataloader, loss)],
    evaluator = dev_evaluator,
    epochs = 5,
    evaluation_steps = 1000,
    warmup_steps = warmup_steps,
    output_path = model_save_path
)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16100 [00:00<?, ?it/s]

In [ ]:
dev_evaluator(model, output_path='/content/drive/MyDrive/Nia_competition')

0.7671465099282297

In [ ]:
model.save('kpfSbert_nli')

In [14]:
from urllib import request, parse
from pprint import pprint
import json
headers = {'Content-Type': 'application/json; chearset=utf-8'}
data = {
    'access_key': 'c623a12a-2a15-44bc-8d93-183b5b6834b4',
    'argument': {
        'query' : '취업',
        'published_at': {
            'from' : '2020-01-01',
            'until' : '2022-08-31'
        },
        'provider' : ['동아일보', '중앙일보'],
        'return_from' : 1001,
        'return_size' : 1000,
        "fields" : ['content']
        }
        }

url = 'http://tools.kinds.or.kr:8888/search/news'
req = request.Request(url, headers=headers, data=json.dumps(data).encode('utf-8'))
res = request.urlopen(req)


json_str = res.read().decode('utf-8')
json_file = json.loads(json_str)


#get the data
data_list = []
for i in range(0, 1000):
  x = json_file['return_object']['documents'][i]['content']
  data_list.append(x)
import pandas as pd

data_preprocessing = pd.Series(data_list)
data_preprocessing = data_preprocessing.str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
data_preprocessing = data_preprocessing.str.replace("\[.*\]|\s-\s.*"," ",regex=True)
data_preprocessing = data_preprocessing.str.replace("\<.*\>|\s-\s.*"," ",regex=True)
data_preprocessing = data_preprocessing.str.replace("무단전재 및 재배포 금지"," ",regex=True)
data_preprocessing = data_preprocessing.str.replace("무단 전재 및 재배포 금지"," ",regex=True)
data_preprocessing = data_preprocessing.str.replace("©"," ",regex=True)
data_preprocessing = data_preprocessing.str.replace("ⓒ"," ",regex=True)
data_preprocessing = data_preprocessing.str.replace("저작권자"," ",regex=True)
data_preprocessing = data_preprocessing.str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
data_preprocessing = data_preprocessing.str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
data_preprocessing = data_preprocessing.str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
data_preprocessing = data_preprocessing.str.replace('\"', "",regex=True)
data_preprocessing = data_preprocessing.str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
data_preprocessing = data_preprocessing.str.replace("\n"," ")
data_preprocessing = data_preprocessing.str.replace("\r"," ")
data_preprocessing = data_preprocessing.str.replace("\t"," ")
data_preprocessing = data_preprocessing.str.replace( "\’" , "", regex=True)
    # data_preprocessing = data_preprocessing.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
data_preprocessing = data_preprocessing.str.replace("[ ]{2,}"," ",regex=True)

In [15]:
#check
data_preprocessing[:3]

0    JTBC 스튜디오가 신규 웹드라마 ‘인턴파서블을 28일 첫 공개한다. ‘인턴파서블은...
1    그리스 여자프로배구 진출을 추진하는 이재영·다영 를 직권으로 발급한 지 13일 만이...
2    서울여자대학교 대학일자리플러스사업단 은 지난 18일 교내 기업현장실습 프로그램인 S...
dtype: object

In [17]:
pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kss-3.6.4.tar.gz (42.4 MB)
  Using cached emoji-1.2.0-py3-none-any.whl (131 kB)
  Created wheel for kss: filename=kss-3.6.4-py3-none-any.whl size=42448613 sha256=4842b200e2f46a4f7dfd260610afe0d4ea70a8deb7ca0dbe02297d2d3fc61ed2
  Stored in directory: /root/.cache/pip/wheels/32/53/7c/76bdf098c2a6c5cd4c4c29648da30ad4793d604314b2aeb26f
Successfully built kss


In [48]:
#check the sentence embedding vector
path = '/content/drive/MyDrive/Nia_competition/kpfSbert_nli'
model = SentenceTransformer(path)
#Input the news sentences
texts = [x for x in data_preprocessing]

t_em = model.encode(texts, convert_to_tensor = True)

q = ['저소득 구직자와 청년에게 1인당 최대 300만원의 구직촉진수당을 주고 취업을 지원하는 국민취업지원제도 참여자 4명 중 3명은 여전히 취업하지 못하는 아쉬운 첫발을 내디뎠다.특히 지난해 제도 참여자 중 취·창업자는 10만 6000명으로 전체 참여자 4명 중 3명은 여전히 취업하지 못한 상태라는 뜻이다. 제도를 통해 취업하거나 본인 희망에 따른 중단 등으로 서비스가 종료된 인원은 총 15만 5000명이다.']
q_em = model.encode(q, convert_to_tensor = True)
cos_score = util.pytorch_cos_sim(q_em, t_em)[0]
cos_score = cos_score.cpu()

#top 5
top_results = np.argpartition(-cos_score, range(5))[0:5]
print('top5')
for i in top_results[0:5]:
  print(texts[i].strip(), "(score: %.4f)" % (cos_score[i]))

top5
“취업이 안 돼 심신이 지쳤어요. 아예 한국을 떠날까 봐요.” 주모 씨 는 4년제 대학을 졸업한 뒤 3년간 기업들에 입사지원서를 넣고 학원을 다니며 취업 준비에 공을 들였다. 하지만 원하는 기업에 입사할 순 없었다. 자신감이 떨어져 점차 지원서를 제출하는 곳이 줄었다. 그는 6개월 전부턴 아예 취업을 포기하고 부모님께 받은 용돈으로 생활하고 있다. 해외로 떠나 새로운 기회를 찾아야 할까 고민이다. 주 씨처럼 구직 활동을 포기한 ‘구직단념자가 지난해 63만 명에 육박하며 사상 최대치를 찍었다. 6개월 이상 직장을 구하지 못한 ‘장기 실업자는 지난해 3년 만에 증가로 전환했다. 이들 중 절반은 20, 30대 청년층이다. 정부는 재정을 투입해 공공 일자리를 늘리며 고용이 회복세에 접어들었다고 평가하지만 전문가들은 구직단념자나 장기 실업자가 늘어나는 점을 심각하게 받아들이고 대응해야 한다고 지적한다. ○ “구직단념자, 경기위기 때 실질적 고용지표” 24일 통계청 국가통계포털에 따르면 지난해 구직단념자는 62만8000명으로 관련 통계가 개편된 2014년 이후 가장 많았다. 신종 코로나바이러스 감염증 에 비해 3.8% 늘었다. 구직단념자는 소득 활동을 하지 않는 비경제활동인구 가운데 지난 1년 이내 구직 경험이 있지만 원하는 일자리가 없어 지금은 구직을 하지 않는 사람이다. 일자리가 없어도 실업자로 집계되진 않아 ‘숨은 실업자인 셈이다. 전문가들은 구직단념자 수가 경제위기 때 실업률이나 고용률보다 일자리 상황을 직접적으로 설명하는 지표라고 평가한다. 실업률이나 고용률은 정부가 재정 투입으로 공공 일자리를 만들면 개선된다. 숨은 실업자가 늘어도 지표는 좋아지니 ‘고용시장이 개선된다는 착시를 줄 수 있는 것이다. 김유빈 한국노동연구원 연구위원은 “코로나19가 극심했던 2020년 4월 15∼29세 실업자는 37만3000명으로 전달보다 오히려 2만9000명 줄어든 착시가 나타났다”며 “구직단념자 추이는 경기위기 때 일자리 상황을 실질적으로 체감할 수 있는 지표”라고 했